In [2]:
import cv2
import numpy as np
import shutil
import os
import mediapipe as mp
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [5]:
mp_holistic = mp.solutions.hands 
mp_drawing = mp.solutions.drawing_utils

In [6]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [7]:
def draw_styled_landmarks(image, results):
    # Draw hands connections
    if results.multi_hand_landmarks:
        for num, hand in enumerate(results.multi_hand_landmarks):
            mp_drawing.draw_landmarks(image, hand, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

In [8]:
def matrx_edt(resultspose):
    mat = []
    for res in resultspose:
        test = np.array([res.x, res.y, res.z])
        mat.append(test)
    return mat
def extract_keypoints(results):
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            lefthand = np.array(matrx_edt(hand_landmarks.landmark)).flatten()
    else: lefthand = np.zeros(21*3)
        
    return np.concatenate([lefthand])

In [9]:
DATA_PATH = os.path.join('database')
actions=np.array([x for x in os.listdir("dataset/train")])

In [7]:

if os.path.exists(DATA_PATH) and os.path.isdir(DATA_PATH):
    shutil.rmtree(DATA_PATH)
    
for action in actions: 
    try:
        os.makedirs(os.path.join(DATA_PATH, action))
    except:
        print("the folders is exist or another Error")
        break

In [8]:
directory = "dataset/train"
with mp_holistic.Hands(static_image_mode=True,min_detection_confidence=0.8, min_tracking_confidence=0.5, ) as hands:
    for folder in os.listdir(directory):
        tmp = 0
        while tmp < 900:
            
            for i,img in enumerate(os.listdir(directory+'/'+folder)):
                train_dir = directory+'/'+folder+'/'+img
                sample_img = cv2.imread(train_dir)
                image, results = mediapipe_detection(sample_img, hands)
                draw_styled_landmarks(image, results)
                keypoints = extract_keypoints(results)
                if results.multi_hand_landmarks or folder == "nothing":
                    npy_path = os.path.join(DATA_PATH, folder, str(tmp))
                    np.save(npy_path, keypoints)
                    tmp+=1
                if tmp >= 900:
                    break

In [10]:
label_map = {label:num for num, label in enumerate(actions)}
DATA_PATH = os.path.join('database')       
sequences, labels = [], []
for action in actions:
    window = []
    for num,sequence in enumerate(os.listdir(DATA_PATH +'/'+ action)):    
        res = np.load(os.path.join(DATA_PATH, action,sequence))
        window.append(res)
        if len(window) % 30 == 0 and num != 0:
            sequences.append(window)
            labels.append(label_map[action])
            window = []

In [11]:
print(np.array(res).shape)
print(np.array(labels).shape)

(63,)
(90,)


In [13]:
x = np.array(sequences)
y = to_categorical(labels).astype(int)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15)

In [14]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [15]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,21*3)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [16]:
model.fit(x_train, y_train, epochs=2000, validation_data=(x_test, y_test), callbacks=[tb_callback])

Epoch 1/2000
3/3 [==============================] - 6s 482ms/step - loss: 1.1002 - categorical_accuracy: 0.3026 - val_loss: 1.0917 - val_categorical_accuracy: 0.5714
Epoch 2/2000
3/3 [==============================] - 0s 44ms/step - loss: 1.0888 - categorical_accuracy: 0.6184 - val_loss: 1.0851 - val_categorical_accuracy: 0.6429
Epoch 3/2000
3/3 [==============================] - 0s 46ms/step - loss: 1.0763 - categorical_accuracy: 0.7763 - val_loss: 1.0584 - val_categorical_accuracy: 0.8571
Epoch 4/2000
3/3 [==============================] - 0s 46ms/step - loss: 1.0415 - categorical_accuracy: 0.8553 - val_loss: 0.9967 - val_categorical_accuracy: 0.7857
Epoch 5/2000
3/3 [==============================] - 0s 57ms/step - loss: 0.8968 - categorical_accuracy: 0.8158 - val_loss: 1.3596 - val_categorical_accuracy: 0.5000
Epoch 6/2000
3/3 [==============================] - 0s 52ms/step - loss: 0.8981 - categorical_accuracy: 0.5658 - val_loss: 0.9755 - val_categorical_accuracy: 0.4286
Epoch 7/2

3/3 [==============================] - 0s 54ms/step - loss: 0.0061 - categorical_accuracy: 1.0000 - val_loss: 0.0671 - val_categorical_accuracy: 0.9286
Epoch 100/2000
3/3 [==============================] - 0s 45ms/step - loss: 0.0069 - categorical_accuracy: 1.0000 - val_loss: 0.0056 - val_categorical_accuracy: 1.0000
Epoch 101/2000
3/3 [==============================] - 0s 48ms/step - loss: 8.7200e-04 - categorical_accuracy: 1.0000 - val_loss: 0.0014 - val_categorical_accuracy: 1.0000
Epoch 102/2000
3/3 [==============================] - 0s 49ms/step - loss: 2.8631e-04 - categorical_accuracy: 1.0000 - val_loss: 4.7813e-04 - val_categorical_accuracy: 1.0000
Epoch 103/2000
3/3 [==============================] - 0s 57ms/step - loss: 3.1849e-04 - categorical_accuracy: 1.0000 - val_loss: 2.4309e-04 - val_categorical_accuracy: 1.0000
Epoch 104/2000
3/3 [==============================] - 0s 57ms/step - loss: 0.0271 - categorical_accuracy: 0.9737 - val_loss: 5.7141e-04 - val_categorical_accura

Epoch 148/2000
3/3 [==============================] - 0s 44ms/step - loss: 0.3027 - categorical_accuracy: 0.9211 - val_loss: 0.3022 - val_categorical_accuracy: 0.8571
Epoch 149/2000
3/3 [==============================] - 0s 46ms/step - loss: 0.3633 - categorical_accuracy: 0.9342 - val_loss: 0.3210 - val_categorical_accuracy: 0.8571
Epoch 150/2000
3/3 [==============================] - 0s 45ms/step - loss: 0.2466 - categorical_accuracy: 0.9211 - val_loss: 0.3329 - val_categorical_accuracy: 0.8571
Epoch 151/2000
3/3 [==============================] - 0s 44ms/step - loss: 0.5825 - categorical_accuracy: 0.8816 - val_loss: 0.5692 - val_categorical_accuracy: 0.7143
Epoch 152/2000
3/3 [==============================] - 0s 44ms/step - loss: 0.5592 - categorical_accuracy: 0.7105 - val_loss: 0.3334 - val_categorical_accuracy: 0.9286
Epoch 153/2000
3/3 [==============================] - 0s 44ms/step - loss: 0.3717 - categorical_accuracy: 0.8816 - val_loss: 0.2341 - val_categorical_accuracy: 1.000

3/3 [==============================] - 0s 42ms/step - loss: 0.0347 - categorical_accuracy: 0.9868 - val_loss: 0.2074 - val_categorical_accuracy: 0.9286
Epoch 198/2000
3/3 [==============================] - 0s 54ms/step - loss: 0.0329 - categorical_accuracy: 0.9868 - val_loss: 0.2282 - val_categorical_accuracy: 0.9286
Epoch 199/2000
3/3 [==============================] - 0s 64ms/step - loss: 0.0342 - categorical_accuracy: 0.9868 - val_loss: 0.2395 - val_categorical_accuracy: 0.9286
Epoch 200/2000
3/3 [==============================] - 0s 57ms/step - loss: 0.0314 - categorical_accuracy: 0.9868 - val_loss: 0.2236 - val_categorical_accuracy: 0.9286
Epoch 201/2000
3/3 [==============================] - 0s 60ms/step - loss: 0.0293 - categorical_accuracy: 0.9868 - val_loss: 0.2189 - val_categorical_accuracy: 0.9286
Epoch 202/2000
3/3 [==============================] - 0s 50ms/step - loss: 0.0277 - categorical_accuracy: 0.9868 - val_loss: 0.2198 - val_categorical_accuracy: 0.9286
Epoch 203/200

Epoch 246/2000
3/3 [==============================] - 0s 47ms/step - loss: 3.2452 - categorical_accuracy: 0.3289 - val_loss: 2.9329 - val_categorical_accuracy: 0.2143
Epoch 247/2000
3/3 [==============================] - 0s 47ms/step - loss: 2.0583 - categorical_accuracy: 0.3553 - val_loss: 1.9465 - val_categorical_accuracy: 0.2143
Epoch 248/2000
3/3 [==============================] - 0s 41ms/step - loss: 1.9648 - categorical_accuracy: 0.3421 - val_loss: 1.1352 - val_categorical_accuracy: 0.3571
Epoch 249/2000
3/3 [==============================] - 0s 46ms/step - loss: 1.5973 - categorical_accuracy: 0.2632 - val_loss: 1.3606 - val_categorical_accuracy: 0.3571
Epoch 250/2000
3/3 [==============================] - 0s 52ms/step - loss: 1.3827 - categorical_accuracy: 0.3421 - val_loss: 1.2867 - val_categorical_accuracy: 0.4286
Epoch 251/2000
3/3 [==============================] - 0s 78ms/step - loss: 1.4817 - categorical_accuracy: 0.2895 - val_loss: 0.9812 - val_categorical_accuracy: 0.428

...

3/3 [==============================] - 0s 125ms/step - loss: 1.1074e-04 - categorical_accuracy: 1.0000 - val_loss: 0.0182 - val_categorical_accuracy: 1.0000
Epoch 1548/2000
3/3 [==============================] - 0s 125ms/step - loss: 1.1053e-04 - categorical_accuracy: 1.0000 - val_loss: 0.0186 - val_categorical_accuracy: 1.0000
Epoch 1549/2000
3/3 [==============================] - 0s 133ms/step - loss: 1.1036e-04 - categorical_accuracy: 1.0000 - val_loss: 0.0189 - val_categorical_accuracy: 1.0000
Epoch 1550/2000
3/3 [==============================] - 0s 132ms/step - loss: 1.1016e-04 - categorical_accuracy: 1.0000 - val_loss: 0.0190 - val_categorical_accuracy: 1.0000
Epoch 1551/2000
3/3 [==============================] - 0s 125ms/step - loss: 1.0999e-04 - categorical_accuracy: 1.0000 - val_loss: 0.0189 - val_categorical_accuracy: 1.0000
Epoch 1552/2000
3/3 [==============================] - 0s 135ms/step - loss: 1.0968e-04 - categorical_accuracy: 1.0000 - val_loss: 0.0190 - val_categor

3/3 [==============================] - 0s 139ms/step - loss: 8.8850e-05 - categorical_accuracy: 1.0000 - val_loss: 0.0168 - val_categorical_accuracy: 1.0000
Epoch 1642/2000
3/3 [==============================] - 0s 148ms/step - loss: 8.8590e-05 - categorical_accuracy: 1.0000 - val_loss: 0.0166 - val_categorical_accuracy: 1.0000
Epoch 1643/2000
3/3 [==============================] - 0s 141ms/step - loss: 8.8390e-05 - categorical_accuracy: 1.0000 - val_loss: 0.0166 - val_categorical_accuracy: 1.0000
Epoch 1644/2000
3/3 [==============================] - 0s 151ms/step - loss: 8.8142e-05 - categorical_accuracy: 1.0000 - val_loss: 0.0168 - val_categorical_accuracy: 1.0000
Epoch 1645/2000
3/3 [==============================] - 0s 146ms/step - loss: 8.7914e-05 - categorical_accuracy: 1.0000 - val_loss: 0.0171 - val_categorical_accuracy: 1.0000
Epoch 1646/2000
3/3 [==============================] - 0s 150ms/step - loss: 8.7822e-05 - categorical_accuracy: 1.0000 - val_loss: 0.0173 - val_categor

3/3 [==============================] - 0s 132ms/step - loss: 7.2409e-05 - categorical_accuracy: 1.0000 - val_loss: 0.0164 - val_categorical_accuracy: 1.0000
Epoch 1736/2000
3/3 [==============================] - 0s 132ms/step - loss: 7.2382e-05 - categorical_accuracy: 1.0000 - val_loss: 0.0162 - val_categorical_accuracy: 1.0000
Epoch 1737/2000
3/3 [==============================] - 0s 132ms/step - loss: 7.2249e-05 - categorical_accuracy: 1.0000 - val_loss: 0.0163 - val_categorical_accuracy: 1.0000
Epoch 1738/2000
3/3 [==============================] - 0s 131ms/step - loss: 7.2110e-05 - categorical_accuracy: 1.0000 - val_loss: 0.0165 - val_categorical_accuracy: 1.0000
Epoch 1739/2000
3/3 [==============================] - 0s 131ms/step - loss: 7.1940e-05 - categorical_accuracy: 1.0000 - val_loss: 0.0166 - val_categorical_accuracy: 1.0000
Epoch 1740/2000
3/3 [==============================] - 0s 140ms/step - loss: 7.1826e-05 - categorical_accuracy: 1.0000 - val_loss: 0.0168 - val_categor

3/3 [==============================] - 0s 120ms/step - loss: 5.9049e-05 - categorical_accuracy: 1.0000 - val_loss: 0.0164 - val_categorical_accuracy: 1.0000
Epoch 1830/2000
3/3 [==============================] - 0s 133ms/step - loss: 5.8850e-05 - categorical_accuracy: 1.0000 - val_loss: 0.0160 - val_categorical_accuracy: 1.0000
Epoch 1831/2000
3/3 [==============================] - 0s 127ms/step - loss: 5.8675e-05 - categorical_accuracy: 1.0000 - val_loss: 0.0156 - val_categorical_accuracy: 1.0000
Epoch 1832/2000
3/3 [==============================] - 0s 142ms/step - loss: 5.8551e-05 - categorical_accuracy: 1.0000 - val_loss: 0.0154 - val_categorical_accuracy: 1.0000
Epoch 1833/2000
3/3 [==============================] - 0s 128ms/step - loss: 5.8348e-05 - categorical_accuracy: 1.0000 - val_loss: 0.0153 - val_categorical_accuracy: 1.0000
Epoch 1834/2000
3/3 [==============================] - 0s 130ms/step - loss: 5.8214e-05 - categorical_accuracy: 1.0000 - val_loss: 0.0151 - val_categor

3/3 [==============================] - 0s 149ms/step - loss: 4.7068e-05 - categorical_accuracy: 1.0000 - val_loss: 0.0162 - val_categorical_accuracy: 1.0000
Epoch 1924/2000
3/3 [==============================] - 0s 141ms/step - loss: 4.6853e-05 - categorical_accuracy: 1.0000 - val_loss: 0.0160 - val_categorical_accuracy: 1.0000
Epoch 1925/2000
3/3 [==============================] - 0s 128ms/step - loss: 4.6701e-05 - categorical_accuracy: 1.0000 - val_loss: 0.0159 - val_categorical_accuracy: 1.0000
Epoch 1926/2000
3/3 [==============================] - 0s 140ms/step - loss: 4.6610e-05 - categorical_accuracy: 1.0000 - val_loss: 0.0158 - val_categorical_accuracy: 1.0000
Epoch 1927/2000
3/3 [==============================] - 0s 143ms/step - loss: 4.6536e-05 - categorical_accuracy: 1.0000 - val_loss: 0.0157 - val_categorical_accuracy: 1.0000
Epoch 1928/2000
3/3 [==============================] - 0s 123ms/step - loss: 4.6532e-05 - categorical_accuracy: 1.0000 - val_loss: 0.0156 - val_categor

In [19]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 30, 64)            32768     
                                                                 
 lstm_4 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_5 (LSTM)               (None, 64)                49408     
                                                                 
 dense_3 (Dense)             (None, 64)                4160      
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 3)                 99        
                                                                 
Total params: 187,331
Trainable params: 187,331
Non-tr

In [20]:
del model

In [21]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,21*3)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.load_weights('action.h5')

In [23]:
sequence = []
sentence = []
threshold = 0.5
text=""

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5) as hands:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, hands)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        if np.all((keypoints == 0)) == False:
            sequence.append(keypoints)
            sequence = sequence[-30:]

            if len(sequence) == 30:
                res = model.predict(np.expand_dims(sequence, axis=0))[0]
                print(actions[np.argmax(res)])
                
                if res[np.argmax(res)] > threshold: 
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

                if len(sentence) > 1: 
                    sentence = sentence[-1:]

        image = cv2.flip(image,1)
       
        if len(sentence)> 0 :
            text = sentence[0]
        # putting the FPS count on the frame
        cv2.putText(image, text, (3,30),cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
   
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

1/1 [==============================] - 0s 38ms/step
rock
1/1 [==============================] - 0s 45ms/step
rock
1/1 [==============================] - 0s 47ms/step
rock
1/1 [==============================] - 0s 22ms/step
rock
1/1 [==============================] - 0s 28ms/step
rock
1/1 [==============================] - 0s 50ms/step
rock
1/1 [==============================] - 0s 44ms/step
rock
1/1 [==============================] - 0s 44ms/step
rock
1/1 [==============================] - 0s 43ms/step
rock
1/1 [==============================] - 0s 55ms/step
rock
1/1 [==============================] - 0s 31ms/step
rock
1/1 [==============================] - 0s 42ms/step
rock
1/1 [==============================] - 0s 50ms/step
rock
1/1 [==============================] - 0s 30ms/step
rock
1/1 [==============================] - 0s 43ms/step
rock
1/1 [==============================] - 0s 24ms/step
rock
1/1 [==============================] - 0s 28ms/step
rock
1/1 [==========================